In [ ]:
print('merge_notifica')
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica, auto_fit_columns
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","merge_notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [ ]:
cc = CasosConfirmados()
cc.load()

notifica = Notifica()
notifica.load()

print(cc.shape())
print(notifica.shape())

In [ ]:
casosc = cc.get_casos()
casosn = notifica.get_casos()
# casosn = casosn.loc[(casosn['duplicado']==0) | ((casosn['duplicado']==1) & (casosn['manter']==1))]
print(casosn.shape)

In [ ]:
idx_casos_confirmados = []
casos_merged = pd.DataFrame(columns=casosc.columns)
for mes_cc, cc, mes_cn, cn in zip(casosc.groupby('data_diagnostico'),casosn.groupby('data_diagnostico')):
    idx_casos_confirmados += cc.loc[cc['hash_resid'].isin(cn['hash_resid'])]
    idx_casos_confirmados += cc.loc[cc['hash_atend'].isin(cn['hash_atend'])]
    idx_casos_confirmados += cc.loc[cc['hash_diag'].isin(cn['hash_diag'])]

    casos_merged = casos_merged.append(
        pd.merge(cc,cn['id','hash_resid'], how='left')
    ).append(
        pd.merge(cc.loc[cc['id'].notnull()].drop(columns='id'),cn['id','hash_atend'], how='left')
    ).append(
        pd.merge(cc.loc[cc['id'].notnull()].drop(columns='id'),cn['id','hash_diag'], how='left')
    )
    

idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.shape

In [ ]:
casos_merged